In [1]:
# Standard imports

# get the environment
import sys
print(sys.version)
print(sys.executable)
import os
os.getcwd()

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
C:\ProgramData\Anaconda3\python.exe


'C:\\Users\\bbutler\\BButlerStuff\\PLoan'

In [2]:
import multiprocessing
cores = multiprocessing.cpu_count()
print(f"There are {cores} cores available.")

There are 16 cores available.


In [3]:
# import base libraries
import pandas as pd
import numpy as np


# set options
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.4f}'.format

# Overview
## This model is designed to take in loan terms (contract amount, term and apr) and then
## returns the financial values of each loan: Monthly Payment, Interest Income,
## Total Costs, Profit and NPV of Profit

# Get the Data and Create a Function to Calculate Monthly Payment

In [31]:
# set the directory
loanDir = 'C:\\Users\\bbutler\\BButlerStuff\\PLoan\\PortSep2021'

os.chdir(loanDir)

# get the file
# bring in balances

fileName = 'PLoanPort2021.csv'

ploanPort = pd.read_csv(fileName,low_memory=False, dtype=str)
ploanPort.head()

,AppId,CustomerNumber,ContractNum,contractAmt,Term,Interest Rate
0,2027,1453250,3750000223,13000,60,9.49
1,2041,00000458159,3750000215,5000,60,9.49
2,2046,00001118347,3750000219,12000,60,9.49
3,2069,00002287661,3750000233,7500,60,9.49
4,2073,00001428711,3750000227,4555,36,9.49


In [32]:
ploanPort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5222 entries, 0 to 5221
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AppId           5222 non-null   object
 1   CustomerNumber  5222 non-null   object
 2   ContractNum     5222 non-null   object
 3   contractAmt     5222 non-null   object
 4   Term            5222 non-null   object
 5   Interest Rate   5222 non-null   object
dtypes: object(6)
memory usage: 244.9+ KB


In [33]:
# change the interest rate 

ploanPort['Interest Rate'] = ploanPort['Interest Rate'].astype(float)/100
ploanPort['Term'] = ploanPort['Term'].astype(int)
ploanPort['contractAmt'] = ploanPort['contractAmt'].astype(int)
ploanPort.columns = ['AppId', 'CustomerNum', 'ContractNum', 'ContractAmt', 'Term', 'APR']
# ploanPort['AppId'] = ploanPort['AppId'].astype(str)
# ploanPort['ContractNum'] = ploanPort['ContractNum'].astype(str)
ploanPort.head()

,AppId,CustomerNum,ContractNum,ContractAmt,Term,APR
0,2027,1453250,3750000223,13000,60,0.0949
1,2041,00000458159,3750000215,5000,60,0.0949
2,2046,00001118347,3750000219,12000,60,0.0949
3,2069,00002287661,3750000233,7500,60,0.0949
4,2073,00001428711,3750000227,4555,36,0.0949


In [34]:
ploanPort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5222 entries, 0 to 5221
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AppId        5222 non-null   object 
 1   CustomerNum  5222 non-null   object 
 2   ContractNum  5222 non-null   object 
 3   ContractAmt  5222 non-null   int32  
 4   Term         5222 non-null   int32  
 5   APR          5222 non-null   float64
dtypes: float64(1), int32(2), object(3)
memory usage: 204.1+ KB


In [35]:
# formula to calculate the payment
def calcPayment(origBalance, term, apr):
    """
    Function calculates the monthly payment for a Loan given the balance, term and apr
    
    term: should be a period in months
    
    
    """
    # break formula into numerator and denominator
    numerator = (origBalance * (apr/12) * (1+(apr/12))**term)
    denominator = (((1+(apr/12))**term) - 1)
    
    monthPayment = numerator/denominator
    
    return monthPayment

In [36]:
# create payment 

ploanPort['CalcPayment'] = ploanPort.apply(lambda df: calcPayment(df.ContractAmt, df.Term, df.APR), axis = 1)
ploanPort.head()

,AppId,CustomerNum,ContractNum,ContractAmt,Term,APR,CalcPayment
0,2027,1453250,3750000223,13000,60,0.0949,272.9607
1,2041,00000458159,3750000215,5000,60,0.0949,104.9849
2,2046,00001118347,3750000219,12000,60,0.0949,251.9637
3,2069,00002287661,3750000233,7500,60,0.0949,157.4773
4,2073,00001428711,3750000227,4555,36,0.0949,145.8888


# Set up Key Inputs & Model Parameters

## The model utilizes two data frames to perform all calculations.
## 1. Input dataframe contains the loan ID, and loan details: contract amt, term and apr
## 2. The second dataframe calculates the details of the loan for each month of the loan term and applies discounting to generate the NPV of the loan.

## The model loops through all of the loans in the 1st dataframe to generate a 2nd temporary dataframe for each loan which is rolled up and the values are transferred back to the 1st dataframe with the rest of the financial resolts.

# Step 1: Calculate Interest (This is the Loan Revenue)

In [37]:
ploanPort['Interest'] = (ploanPort['CalcPayment']*ploanPort['Term']) - ploanPort['ContractAmt']
ploanPort.head()

,AppId,CustomerNum,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest
0,2027,1453250,3750000223,13000,60,0.0949,272.9607,3377.6403
1,2041,00000458159,3750000215,5000,60,0.0949,104.9849,1299.0924
2,2046,00001118347,3750000219,12000,60,0.0949,251.9637,3117.8218
3,2069,00002287661,3750000233,7500,60,0.0949,157.4773,1948.6386
4,2073,00001428711,3750000227,4555,36,0.0949,145.8888,696.9966


# Build out the 2nd dataframe of loan details (revenue and costs)

In [38]:
# drop AppID
df = ploanPort[['AppId', 'CustomerNum']].copy()
df['AppId'] = df['AppId'].astype('str')
df.head()


,AppId,CustomerNum
0,2027,1453250
1,2041,00000458159
2,2046,00001118347
3,2069,00002287661
4,2073,00001428711


In [40]:
ploanPort = ploanPort.drop(['AppId', 'CustomerNum'], axis = 1)
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest
0,3750000223,13000,60,0.0949,272.9607,3377.6403
1,3750000215,5000,60,0.0949,104.9849,1299.0924
2,3750000219,12000,60,0.0949,251.9637,3117.8218
3,3750000233,7500,60,0.0949,157.4773,1948.6386
4,3750000227,4555,36,0.0949,145.8888,696.9966


In [41]:
numLoans = len(ploanPort)
numLoans

5222

In [42]:
# key parameters
numLoans = len(ploanPort)
ln = 0

while ln < numLoans:

    ########################  MODEL INPUTS FROM THE CSV
    monthly = ploanPort['CalcPayment'][ln]
    orig = ploanPort['ContractAmt'][ln]
    term = ploanPort['Term'][ln].astype(int)
    rate = ploanPort['APR'][ln]


    ######################  COST PARAMS TO BE PROVIDED BY PRODUCT
    # inputs for loan costs
    costOfFunds = .025
    provisions = .01
    servicingCosts = 13.34
    nplRate = .0025
    originationCosts = 54.72
    earlyPayoff = .0043
    docSupport = 19.56
    discountRate = .10

    #############    BEGIN THE DATAFRAME BUILD
    # populate period for first loan
    periods = np.array(range(0,term + 1))

    # create the empty data frame
    dict = {'Period': periods}

    loanDf = pd.DataFrame(dict)

    # fill in the monthly
    loanDf['MonthlyPayment'] = monthly

    # zero out monthly payment for first month
    loanDf.loc[0, 'MonthlyPayment'] = 0

    # get the beginning balance
    loanDf['BeginBal'] = orig

    # now iterate through to generate outstanding balance for each subsequent term
    loanDf.loc[0, 'OutBal'] = loanDf.loc[0, 'BeginBal'] - loanDf.loc[0, 'MonthlyPayment']

    for i in range(1, len(loanDf)):
        loanDf.loc[i, 'BeginBal'] = loanDf.loc[i - 1, 'OutBal']
        loanDf.loc[i, 'OutBal'] = loanDf.loc[i, 'BeginBal'] * (1 + rate/12) - loanDf.loc[i, 'MonthlyPayment']

    # then drop row zero will only need to calculate for terms 1 to n (end)
    loanDf = loanDf.drop(0)

    # create interest income
    loanDf['IntIncome'] = (loanDf['BeginBal']*(1+rate/12))-loanDf['BeginBal']


    # create all of the cost columns from the inputs
    # append them all to a list
    loanDf['CostOfFunds'] = loanDf['OutBal'] * costOfFunds/12
    loanDf['Provisions'] = loanDf['OutBal'] * provisions/12
    loanDf['ServicingCosts'] = loanDf['Period'].apply(lambda x: servicingCosts if x < term else 0)
    loanDf['OriginationCosts'] = originationCosts/term
    loanDf['DocSupport'] = docSupport/term
    loanDf['EarlyPayment'] = loanDf['OutBal'] * earlyPayoff/12


    # select the columns to add up for costs
    costCols = ['CostOfFunds', 'Provisions', 'ServicingCosts',
             'OriginationCosts', 'DocSupport', 'EarlyPayment']

    # make a total cost by summing up the cost columns
    loanDf['TotalCost'] = loanDf.loc[:,costCols].sum(axis=1)

    # calculate undiscounted profit/loss
    loanDf['Profit'] = loanDf['IntIncome'] - loanDf['TotalCost']

    # calculate NPV for the loan at each term
    loanDf['NPV'] = loanDf['Profit']/((1+discountRate/12)**loanDf['Period'])


    # roll it all up into a new aggregate dataframe
    loanTotal = loanDf.groupby(['MonthlyPayment'],
                            as_index=False).agg({'IntIncome': np.sum,
                                                 'TotalCost': np.sum,
                                                 'Profit': np.sum,
                                                 'NPV': np.sum})


    # set the columns to append last three: TotalCost, Profit, NPV
    cols = loanTotal.columns[2:].values.tolist()

    # create the colums in the original dataframe
    ploanPort = pd.concat([ploanPort,pd.DataFrame(columns=list(cols))])

    # set the columns to append last three: TotalCost, Profit, NPV
    cols = loanTotal.columns[2:].values.tolist()

    # create the colums in the original dataframe
    ploanPort = pd.concat([ploanPort,pd.DataFrame(columns=list(cols))])
    ploanPort.head()

    # put this in the porfolio DB
    j = 1

    for i in range(2,5):
        ploanPort.iloc[ln,j+5] = loanTotal.iloc[0,i]
        j += 1
    
    # go to the next loan
    ln +=1
    
    

# Get the Final Update Worksheet

In [43]:
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,3750000223,13000.0000,60.0000,0.0949,272.9607,3377.6403,2217.5138,1160.1265,1022.7960
1,3750000215,5000.0000,60.0000,0.0949,104.9849,1299.0924,1382.9453,-83.8529,-23.8545
2,3750000219,12000.0000,60.0000,0.0949,251.9637,3117.8218,2113.1928,1004.6290,891.9647
3,3750000233,7500.0000,60.0000,0.0949,157.4773,1948.6386,1643.7480,304.8907,303.2238
4,3750000227,4555.0000,36.0000,0.0949,145.8888,696.9966,814.9027,-117.9061,-87.0421


In [44]:
# bind app id
ploanPort['AppId'] = df['AppId'].copy()
ploanPort['CustomerNum'] = df['CustomerNum'].copy()
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV,AppId,CustomerNum
0,3750000223,13000.0000,60.0000,0.0949,272.9607,3377.6403,2217.5138,1160.1265,1022.7960,2027,1453250
1,3750000215,5000.0000,60.0000,0.0949,104.9849,1299.0924,1382.9453,-83.8529,-23.8545,2041,00000458159
2,3750000219,12000.0000,60.0000,0.0949,251.9637,3117.8218,2113.1928,1004.6290,891.9647,2046,00001118347
3,3750000233,7500.0000,60.0000,0.0949,157.4773,1948.6386,1643.7480,304.8907,303.2238,2069,00002287661
4,3750000227,4555.0000,36.0000,0.0949,145.8888,696.9966,814.9027,-117.9061,-87.0421,2073,00001428711


In [47]:
finalPort = ploanPort[['AppId', 'CustomerNum', 'ContractNum', 'ContractAmt','Term', 'APR', 'CalcPayment', 'Interest',
       'TotalCost', 'Profit', 'NPV']]
finalPort.head()

,AppId,CustomerNum,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,2027,1453250,3750000223,13000.0000,60.0000,0.0949,272.9607,3377.6403,2217.5138,1160.1265,1022.7960
1,2041,00000458159,3750000215,5000.0000,60.0000,0.0949,104.9849,1299.0924,1382.9453,-83.8529,-23.8545
2,2046,00001118347,3750000219,12000.0000,60.0000,0.0949,251.9637,3117.8218,2113.1928,1004.6290,891.9647
3,2069,00002287661,3750000233,7500.0000,60.0000,0.0949,157.4773,1948.6386,1643.7480,304.8907,303.2238
4,2073,00001428711,3750000227,4555.0000,36.0000,0.0949,145.8888,696.9966,814.9027,-117.9061,-87.0421


In [48]:
finalPort.shape

(5222, 11)

In [49]:
# change contractNum to AccountNum
newName = {'ContractNum': 'AccountNum'}
finalPort.rename(columns=newName, inplace=True)
finalPort.head()

,AppId,CustomerNum,AccountNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,2027,1453250,3750000223,13000.0000,60.0000,0.0949,272.9607,3377.6403,2217.5138,1160.1265,1022.7960
1,2041,00000458159,3750000215,5000.0000,60.0000,0.0949,104.9849,1299.0924,1382.9453,-83.8529,-23.8545
2,2046,00001118347,3750000219,12000.0000,60.0000,0.0949,251.9637,3117.8218,2113.1928,1004.6290,891.9647
3,2069,00002287661,3750000233,7500.0000,60.0000,0.0949,157.4773,1948.6386,1643.7480,304.8907,303.2238
4,2073,00001428711,3750000227,4555.0000,36.0000,0.0949,145.8888,696.9966,814.9027,-117.9061,-87.0421


In [50]:
# write to csv
finalPort.to_csv('ploanPort_2021Q1Booked.csv', index=False)

In [52]:
os.getcwd()

'C:\\Users\\bbutler\\BButlerStuff\\PLoan\\PortSep2021'